In [ ]:
import import_ipynb
import os
import cv2
import numpy as np
import math
import random
import PossibleChar
import main
import Hog_Algo
import glob 
from sklearn import svm
import pickle


In [ ]:
#Module level Variables:

 # constants for checkIfPossibleChar, this checks one possible char only (does not compare to another char)
MIN_PIXEL_WIDTH = 2
MIN_PIXEL_HEIGHT = 6

MIN_ASPECT_RATIO = 0.25
MAX_ASPECT_RATIO = 1.0

MIN_PIXEL_AREA = 80

        # constants for comparing two chars
MIN_DIAG_SIZE_MULTIPLE_AWAY = 0.3
MAX_DIAG_SIZE_MULTIPLE_AWAY = 5.0

MAX_CHANGE_IN_AREA = 0.5

MAX_CHANGE_IN_WIDTH = 0.8
MAX_CHANGE_IN_HEIGHT = 0.2

MAX_ANGLE_BETWEEN_CHARS = 12.0

        # other constants
MIN_NUMBER_OF_MATCHING_CHARS = 5

RESIZED_CHAR_IMAGE_WIDTH = 20
RESIZED_CHAR_IMAGE_HEIGHT = 30

MIN_CONTOUR_AREA = 80

SCALAR_BLACK = (0.0, 0.0, 0.0)
SCALAR_WHITE = (255.0, 255.0, 255.0)
SCALAR_YELLOW = (0.0, 255.0, 255.0)
SCALAR_GREEN = (0.0, 255.0, 0.0)
SCALAR_RED = (0.0, 0.0, 255.0)

In [ ]:
def checkIfPossibleChar(possibleChar):
            # this function is a 'first pass' that does a rough check on a contour to see if it could be a char,
            # note that we are not (yet) comparing the char to other chars to look for a group
    if (possibleChar.intBoundingRectArea > MIN_PIXEL_AREA and
        possibleChar.intBoundingRectWidth > MIN_PIXEL_WIDTH and possibleChar.intBoundingRectHeight > MIN_PIXEL_HEIGHT and
        MIN_ASPECT_RATIO < possibleChar.fltAspectRatio and possibleChar.fltAspectRatio < MAX_ASPECT_RATIO):
        return True
    else:
        return False

In [ ]:
def findList_MatchingChars(list_possible_chars):
    List_matchingChars=[]
    for poschar in list_possible_chars:
        matching_chars=find_matching_chars(poschar,list_possible_chars)
        matching_chars.append(poschar)
        if len(matching_chars) < MIN_NUMBER_OF_MATCHING_CHARS:
            continue
        List_matchingChars.append(matching_chars)
        
        remaining_list=[]
        remaining_list=list(set(list_possible_chars)-set(matching_chars))
        
        remaining_matches = findList_MatchingChars(remaining_list)      # recursive call
        
        for list_match in remaining_matches:        # for each list of matching chars found by recursive call
            List_matchingChars.append(list_match)             # add to our original list of lists of matching chars
        # end for

        break       # exit for

    # end for

    return List_matchingChars

In [ ]:
def find_matching_chars(poschar,list_possible_chars):
    listOfMatchingChars = []
    for pos in list_possible_chars:
        if pos == poschar:
            # if the char we attempting to find matches for is the exact same char as the char in the big list we are currently checking
            # then we should not include it in the list of matches b/c that would end up double including the current char
            continue
        dist_bw_chars=distchars(poschar,pos)  
        angle_bw_chars=anglechars(poschar,pos)
        ChangeInArea = float(abs(poschar.intBoundingRectArea - pos.intBoundingRectArea)) / float(poschar.intBoundingRectArea)
        ChangeInWidth = float(abs(poschar.intBoundingRectWidth - pos.intBoundingRectWidth)) / float(poschar.intBoundingRectWidth)
        ChangeInHeight = float(abs(poschar.intBoundingRectHeight - pos.intBoundingRectHeight)) / float(poschar.intBoundingRectHeight)
        if (dist_bw_chars < (poschar.fltDiagonalSize * MAX_DIAG_SIZE_MULTIPLE_AWAY) and
            angle_bw_chars < MAX_ANGLE_BETWEEN_CHARS and
            ChangeInArea < MAX_CHANGE_IN_AREA and
            ChangeInWidth < MAX_CHANGE_IN_WIDTH and
            ChangeInHeight < MAX_CHANGE_IN_HEIGHT):

            listOfMatchingChars.append(pos)        # if the chars are a match, add the current char to list of matching chars
        # end if
    # end for

    return listOfMatchingChars                  # return result
# end function

In [ ]:
def anglechars(firstChar,secondChar):
    Adj = float(abs(firstChar.intCenterX - secondChar.intCenterX))
    Opp = float(abs(firstChar.intCenterY - secondChar.intCenterY))
    if Adj != 0.0:                           # check to make sure we do not divide by zero if the center X positions are equal, float division by zero will cause a crash in Python
        AngleInRad = math.atan(Opp / Adj)      # if adjacent is not zero, calculate angle
    else:
        AngleInRad = 1.5708                          # if adjacent is zero, use this as the angle, this is to be consistent with the C++ version of this program
    AngleInDeg = AngleInRad * (180.0 / math.pi)       # calculate angle in degrees

    return AngleInDeg
    # end if

In [ ]:
def distchars(firstChar,secondChar):
    X = abs(firstChar.intCenterX - secondChar.intCenterX)
    Y = abs(firstChar.intCenterY - secondChar.intCenterY)
    return math.sqrt((X ** 2) + (Y ** 2))

In [ ]:
def detectCharsInPlates(listOfPossiblePlates):
    intPlateCounter = 0
    imgContours = None
    contours = []
    
    #Step1: check if the list is empty
    if len(listOfPossiblePlates) == 0:
        return listOfPossiblePlates
    
    #Step2: Processing each plate to extract the characters
    for possiblePlate in listOfPossiblePlates: 
        
        possiblePlate.imgGrayscale, possiblePlate.imgThresh = main.preprocess(possiblePlate.imgPlate)
        #cv2.imshow("5a", possiblePlate.imgPlate)
        #cv2.imshow("5b", possiblePlate.imgGrayscale)
        #cv2.imshow("5c", possiblePlate.imgThresh)
        
        possiblePlate.imgThresh = cv2.resize(possiblePlate.imgThresh, (0, 0), fx = 1.6, fy = 1.6)
        thresholdValue, possiblePlate.imgThresh = cv2.threshold(possiblePlate.imgThresh, 0.0, 255.0, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        #cv2.imshow("5d", possiblePlate.imgThresh)
        listOfPossibleCharsInPlate = findPossibleCharsInPlate(possiblePlate.imgGrayscale, possiblePlate.imgThresh)
        
        height, width, numChannels = possiblePlate.imgPlate.shape
        imgContours = np.zeros((height, width, 3), np.uint8)
        del contours[:]
        
        for possibleChar in listOfPossibleCharsInPlate:
                contours.append(possibleChar.contour)
        
        cv2.drawContours(imgContours, contours, -1, SCALAR_WHITE)
        #cv2.imshow("6", imgContours)   
        listOfListsOfMatchingCharsInPlate = findList_MatchingChars(listOfPossibleCharsInPlate)
        
        imgContours = np.zeros((height, width, 3), np.uint8)
        del contours[:]
        
        for listOfMatchingChars in listOfListsOfMatchingCharsInPlate:
            intRandomBlue = random.randint(0, 255)
            intRandomGreen = random.randint(0, 255)
            intRandomRed = random.randint(0, 255)
            for matchingChar in listOfMatchingChars:
                contours.append(matchingChar.contour)
            cv2.drawContours(imgContours, contours, -1, (intRandomBlue, intRandomGreen, intRandomRed))
            #cv2.imshow("Characters", imgContours)
        if (len(listOfListsOfMatchingCharsInPlate) == 0):
            intPlateCounter = intPlateCounter + 1
            possiblePlate.strChars = ""
            continue
                    
        for i in range(0, len(listOfListsOfMatchingCharsInPlate)):                              # within each list of matching chars
            listOfListsOfMatchingCharsInPlate[i].sort(key = lambda matchingChar: matchingChar.intCenterX)        # sort chars from left to right
            listOfListsOfMatchingCharsInPlate[i] = removeInnerOverlappingChars(listOfListsOfMatchingCharsInPlate[i])
        imgContours = np.zeros((height, width, 3), np.uint8)
        for listOfMatchingChars in listOfListsOfMatchingCharsInPlate:
            intRandomBlue = random.randint(0, 255)
            intRandomGreen = random.randint(0, 255)
            intRandomRed = random.randint(0, 255)
            
            del contours[:]
            for matchingChar in listOfMatchingChars:
                contours.append(matchingChar.contour)
            cv2.drawContours(imgContours, contours, -1, (intRandomBlue, intRandomGreen, intRandomRed))
        #cv2.imshow("Char counters", imgContours)
        
        intLenOfLongestListOfChars = 0
        intIndexOfLongestListOfChars = 0
        
        for i in range(0, len(listOfListsOfMatchingCharsInPlate)):
            if len(listOfListsOfMatchingCharsInPlate[i]) > intLenOfLongestListOfChars:
                intLenOfLongestListOfChars = len(listOfListsOfMatchingCharsInPlate[i])
                intIndexOfLongestListOfChars = i
                
        longestListOfMatchingCharsInPlate = listOfListsOfMatchingCharsInPlate[intIndexOfLongestListOfChars]
        imgContours = np.zeros((height, width, 3), np.uint8)
        del contours[:]
        for matchingChar in longestListOfMatchingCharsInPlate:
            contours.append(matchingChar.contour)
        cv2.drawContours(imgContours, contours, -1, SCALAR_WHITE)
        #cv2.imshow("9", imgContours)
        
        possiblePlate.strChars = recognizeCharsInPlate(possiblePlate.imgThresh, longestListOfMatchingCharsInPlate)
        #print("chars found in plate number " + str(
        #        intPlateCounter) + " = " + possiblePlate.strChars + ", click on any image and press a key to continue . . .")
        intPlateCounter = intPlateCounter + 1
        #cv2.waitKey(0)
        
        print("\nchar detection complete\n")
        #cv2.waitKey(0)
        
        return listOfPossiblePlates


In [ ]:
def recognizeCharsInPlate(imgThresh, listOfMatchingChars):
    strChars = ""               # this will be the return value, the chars in the lic plate

    height, width = imgThresh.shape

    imgThreshColor = np.zeros((height, width, 3), np.uint8)

    listOfMatchingChars.sort(key = lambda matchingChar: matchingChar.intCenterX)        # sort chars from left to right
     
    g_images=[]    
    cv2.cvtColor(imgThresh, cv2.COLOR_GRAY2BGR, imgThreshColor)                     # make color version of threshold image so we can draw contours in color on it

    for currentChar in listOfMatchingChars:                                         # for each char in plate
        pt1 = (currentChar.intBoundingRectX, currentChar.intBoundingRectY)
        pt2 = ((currentChar.intBoundingRectX + currentChar.intBoundingRectWidth), (currentChar.intBoundingRectY + currentChar.intBoundingRectHeight))

        cv2.rectangle(imgThreshColor, pt1, pt2, SCALAR_GREEN, 2)           # draw green box around the char

                # crop char out of threshold image
        imgROI = imgThresh[currentChar.intBoundingRectY : currentChar.intBoundingRectY + currentChar.intBoundingRectHeight,
                           currentChar.intBoundingRectX : currentChar.intBoundingRectX + currentChar.intBoundingRectWidth]
 
        #Step 2: Deriving the feature vector for the image :Preprocessing of the image
        width = 160
        height =160
    
        #step-2(a):Image resize to generalize
        dim = (width, height)
        image = cv2.resize(imgROI, dim, interpolation = cv2.INTER_AREA)
       #step-2(b):Coversion to grayscale
        #gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        gray_image= image
        #step-2(c):Canny Edge Detection
        min_val=np.mean(gray_image)*75/100
        max_val=np.mean(gray_image)*125/100
        canny_image=cv2.Canny(gray_image,min_val,max_val)
        ret, thresh = cv2.threshold(canny_image, 127, 255, 0)
        _, contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        area=[]
        for c in contours:
            area.append(cv2.contourArea(c))
        maxpos = area.index(max(area))
        big=contours[maxpos]     
        x,y,w,h = cv2.boundingRect(big)
        box_img=cv2.rectangle(canny_image,(x,y),(x+w,y+h),(0,255,0),2)
        img=box_img
        width = 160
        height =160
        dim = (width, height)
        image = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
        
        #step-2(d):Append processed images to g_images
        g_images.append(image)
    
    #step-3:Recognizing Alphanumeric Characters
    cv2.imshow("Chars",imgThreshColor)
    #step-3(a):Histogram of gradients (importing functions from Hog_Algo)
    No_of_x_partitions = 4
    No_of_y_partitions = 4
    Chars_fv = []

    for image in g_images:
        num = []
        #Step-HOG(a): Slicing
        partition = Hog_Algo.partitions(image,No_of_x_partitions,No_of_y_partitions)
        #Step-HOG(b): Filter
        for x in partition:
            p = Hog_Algo.apply_filter(x)
            for x in p:
                num.append(x)
        #Step-HOG(c): Creation of Feature vector
        Chars_fv.append(num)
     
    #Step 4:Predicting the Alpha-Numeric Characters
    #Prediction usinf SVM model
    Alpha='C:/Users/Winni/Desktop/final Mini/AlphaNumericRecog/svm_model_alpha.sav'
    classifier_alpha = pickle.load(open(Alpha, 'rb'))
    Num='C:/Users/Winni/Desktop/final Mini/AlphaNumericRecog/svm_model_num.sav'
    classifier_num = pickle.load(open(Num, 'rb'))
    Chars_alpha=[]
    Chars_num=[]
    #appending all alphabets
    Chars_alpha.append(Chars_fv[0])
    Chars_alpha.append(Chars_fv[1])
    Chars_alpha.append(Chars_fv[4])
    Chars_alpha.append(Chars_fv[5])
    #appending all numbers
    Chars_num.append(Chars_fv[2])
    Chars_num.append(Chars_fv[3])
    Chars_num.append(Chars_fv[6])
    Chars_num.append(Chars_fv[7])
    Chars_num.append(Chars_fv[8])
    Chars_num.append(Chars_fv[9])
    test_alpha=np.array(Chars_alpha)
    test_num=np.array(Chars_num)
    predict_alpha = classifier_alpha.predict(test_alpha)
    predict_num = classifier_num.predict(test_num)
    strChars_svm=""
    strChars_svm=strChars_svm+str(predict_alpha[0])+str(predict_alpha[1])+str(predict_num[0])+str(predict_num[1])+str(predict_alpha[2])+str(predict_alpha[3])+str(predict_num[2])+str(predict_num[3])+str(predict_num[4])+str(predict_num[5])                       # append current char to full string
    
    
    #prediction using KNN model
    Alpha='C:/Users/Winni/Desktop/final Mini/AlphaNumericRecog/knn_model_alpha.sav'
    classifier_alpha = pickle.load(open(Alpha, 'rb'))
    Num='C:/Users/Winni/Desktop/final Mini/AlphaNumericRecog/knn_model_num.sav'
    classifier_num = pickle.load(open(Num, 'rb'))
    Chars_alpha=[]
    Chars_num=[]
    #appending all alphabets
    Chars_alpha.append(Chars_fv[0])
    Chars_alpha.append(Chars_fv[1])
    Chars_alpha.append(Chars_fv[4])
    Chars_alpha.append(Chars_fv[5])
    #appending all numbers
    Chars_num.append(Chars_fv[2])
    Chars_num.append(Chars_fv[3])
    Chars_num.append(Chars_fv[6])
    Chars_num.append(Chars_fv[7])
    Chars_num.append(Chars_fv[8])
    Chars_num.append(Chars_fv[9])
    test_alpha=np.array(Chars_alpha)
    test_num=np.array(Chars_num)
    predict_alpha = classifier_alpha.predict(test_alpha)
    predict_num = classifier_num.predict(test_num)
    strChars_knn=""
    strChars_knn=strChars_svm+ str(predict_alpha[0])+str(predict_alpha[1])+str(predict_num[0])+str(predict_num[1])+str(predict_alpha[2])+str(predict_alpha[3])+str(predict_num[2])+str(predict_num[3])+str(predict_num[4])+str(predict_num[5])  
     # append current char to full string
    
    return strChars_knn
# end function


In [ ]:
def removeInnerOverlappingChars(listOfMatchingChars):
    listOfMatchingCharsWithInnerCharRemoved = list(listOfMatchingChars)                # this will be the return value

    for currentChar in listOfMatchingChars:
        for otherChar in listOfMatchingChars:
            if currentChar != otherChar:        # if current char and other char are not the same char . . .
                                                                            # if current char and other char have center points at almost the same location . . .
                if distchars(currentChar, otherChar) < (currentChar.fltDiagonalSize * MIN_DIAG_SIZE_MULTIPLE_AWAY):
                                # if we get in here we have found overlapping chars
                                # next we identify which char is smaller, then if that char was not already removed on a previous pass, remove it
                    if currentChar.intBoundingRectArea < otherChar.intBoundingRectArea:         # if current char is smaller than other char
                        if currentChar in listOfMatchingCharsWithInnerCharRemoved:              # if current char was not already removed on a previous pass . . .
                            listOfMatchingCharsWithInnerCharRemoved.remove(currentChar)         # then remove current char
                        # end if
                    else:                                                                       # else if other char is smaller than current char
                        if otherChar in listOfMatchingCharsWithInnerCharRemoved:                # if other char was not already removed on a previous pass . . .
                            listOfMatchingCharsWithInnerCharRemoved.remove(otherChar)           # then remove other char
                        # end if
                    # end if
                # end if
            # end if
        # end for
    # end for

    return listOfMatchingCharsWithInnerCharRemoved
# end function



In [ ]:
def removeInnerOverlappingChars(listOfMatchingChars):
    listOfMatchingCharsWithInnerCharRemoved = list(listOfMatchingChars)                # this will be the return value

    for currentChar in listOfMatchingChars:
        for otherChar in listOfMatchingChars:
            if currentChar != otherChar:        # if current char and other char are not the same char . . .
                                                                            # if current char and other char have center points at almost the same location . . .
                if distchars(currentChar, otherChar) < (currentChar.fltDiagonalSize * MIN_DIAG_SIZE_MULTIPLE_AWAY):
                                # if we get in here we have found overlapping chars
                                # next we identify which char is smaller, then if that char was not already removed on a previous pass, remove it
                    if currentChar.intBoundingRectArea < otherChar.intBoundingRectArea:         # if current char is smaller than other char
                        if currentChar in listOfMatchingCharsWithInnerCharRemoved:              # if current char was not already removed on a previous pass . . .
                            listOfMatchingCharsWithInnerCharRemoved.remove(currentChar)         # then remove current char
                        # end if
                    else:                                                                       # else if other char is smaller than current char
                        if otherChar in listOfMatchingCharsWithInnerCharRemoved:                # if other char was not already removed on a previous pass . . .
                            listOfMatchingCharsWithInnerCharRemoved.remove(otherChar)           # then remove other char
                        # end if
                    # end if
                # end if
            # end if
        # end for
    # end for

    return listOfMatchingCharsWithInnerCharRemoved
# end function



In [ ]:
def findPossibleCharsInPlate(imgGrayscale, imgThresh):
    listOfPossibleChars = []                        # this will be the return value
    contours = []
    imgThreshCopy = imgThresh.copy()

            # find all contours in plate
    imgContours, contours, npaHierarchy = cv2.findContours(imgThreshCopy, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:                        # for each contour
        possibleChar = PossibleChar.PossibleChar(contour)

        if checkIfPossibleChar(possibleChar):              # if contour is a possible char, note this does not compare to other chars (yet) . . .
            listOfPossibleChars.append(possibleChar)       # add to list of possible chars
        # end if
    # end if

    return listOfPossibleChars
